In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
import pickle


# Step 1: Load the dataset using pandas
def load_dataset(filename):
    data = pd.read_csv(filename)
    return data

# Step 2: Function to count and describe NaN values in the dataset
def describe_nan_values(data):
    nan_counts = data.isnull().sum()
    nan_description = data.describe(include='all')
    return nan_counts, nan_description

# Step 3: Function to visualize attributes with histograms
def visualize_histograms(data):
    numeric_columns = data.select_dtypes(include=np.number)
    for col in numeric_columns.columns:
        sns.histplot(data[col], kde=True)
        plt.title(f'Histogram of {col}')
        plt.show()

# Step 4: Split the dataset into train, cross-validation, and test sets
def split_dataset(data):
    X = data.iloc[:, :-1]  # Features (all columns except the last one)
    y = data['Class']      # Target variable

    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
    X_cv, X_test, y_cv, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

    return X_train, y_train, X_cv, y_cv, X_test, y_test

# Step 5: Normalize/Scale the input data using mean and variance
def normalize_data(X_train, X_cv, X_test):
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_cv_scaled = scaler.transform(X_cv)
    X_test_scaled = scaler.transform(X_test)
    # Save the scaler to a file using pickle
    with open("scaler.pkl", 'wb') as scaler_file:
      pickle.dump(scaler, scaler_file)
    return X_train_scaled, X_cv_scaled, X_test_scaled

# Step 6: Train the model with different learning rates and select the best hyperparameters
def train_model(X_train, y_train, learning_rates):
    best_lr = None
    best_score = 0

    for lr in learning_rates:
        model = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=10)
        scores = cross_val_score(model, X_train, y_train, cv=5)
        avg_score = np.mean(scores)
        if avg_score > best_score:
            best_score = avg_score
            best_lr = lr

    return best_lr

# Step 7: Train the final model with the best hyperparameters
def train_final_model(X_train, y_train, best_lr):
    model = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=10)
    model.fit(X_train, y_train)
    return model

# Step 8: Evaluate the model using cross-validation data
def evaluate_model(model, X_cv, y_cv):
    cv_score = model.score(X_cv, y_cv)
    return cv_score

# Step 9: Test the model with the test dataset
def test_model(model, X_test, y_test):
    test_score = model.score(X_test, y_test)
    return test_score


# Step 10: Save the trained model using pickle
def save_model(model, filename):
    with open(filename, 'wb') as file:
        pickle.dump(model, file)


if __name__ == "__main__":
    # Step 1: Load the dataset
    dataset = load_dataset("pulsar.csv")

    # Step 2: Describe NaN values in the dataset
    nan_counts, nan_description = describe_nan_values(dataset)
    print("\nNaN Counts:")
    print(nan_counts)
    print("\nNaN Description:")
    print(nan_description)

    # Step 3: Visualize attributes with histograms
    visualize_histograms(dataset)

    # Step 4: Split the dataset
    X_train, y_train, X_cv, y_cv, X_test, y_test = split_dataset(dataset)

    # Step 5: Normalize the data
    X_train_scaled, X_cv_scaled, X_test_scaled = normalize_data(X_train, X_cv, X_test)

    # Step 6: Train the model with different learning rates
    learning_rates = [0.001, 0.01, 0.1, 1]
    best_lr = train_model(X_train_scaled, y_train, learning_rates)

    # Step 7: Train the final model with the best hyperparameters
    final_model = train_final_model(X_train_scaled, y_train, best_lr)

    # Step 8: Evaluate the model using cross-validation data
    cv_score = evaluate_model(final_model, X_cv_scaled, y_cv)
    print(f"Cross-validation score: {cv_score}")

    # Step 9: Test the model with the test dataset
    test_score = test_model(final_model, X_test_scaled, y_test)
    print(f"Test score: {test_score}")

    # Step 10: Save the trained model
    save_model(final_model, "pulsar_model.pkl")


